# Berlin Data

Time series for the Havel River inflow to the city (Konradshöhe, Messstellennummer 305) and the downstream station (Schleuse Spandau, Messstellennummer 320), DOC and TOC.

Two groundwater station are attached, only with quality, no DOC/TOC is measured here, but UV254 and other. The groundwater stations are not influenced by bank filtrate and represent near-natural conditions (for a city like Berlin).

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
from googletrans import Translator

# Define Paths

In [ ]:
data_folder = os.path.join("..", "..", "data", "berlin")

raw_data_folder = os.path.join(data_folder, "raw_data")
data_info_folder = os.path.join(data_folder, "data_info")

ground_water_folder = os.path.join(raw_data_folder, "ground water")
surface_water_folder = os.path.join(raw_data_folder, "surface water")

# Load Data

## Ground Water

In [ ]:
ts_gw_df = pd.read_csv(
    os.path.join(
        ground_water_folder, "time-series_ground-water_quality.csv"
    )
)

In [ ]:
ts_gw_df

In [ ]:
ts_gw_df.rename(
    columns={
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Messwert": "Value",
    },
    inplace=True,
)

## Surface Water

In [ ]:
ts_sw_df = pd.read_csv(
    os.path.join(
        surface_water_folder, "time-series_surface-water_quality.csv"
    )
)

In [ ]:
flow_df = pd.read_csv(
    os.path.join(
        surface_water_folder, "time-series_surface-water_flow.csv"
    )
)

In [ ]:
ts_sw_df

In [ ]:
ts_sw_df.rename(
    columns={
        "Messstelle": "Station",
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Wert": "Value",
    },
    inplace=True,
)

ts_sw_df.drop(
    columns=[
        'Entnahmetiefe [m]',
        'Vorzeichen',
        'Bestimmungsgrenze',
        'Messmethode'
    ],
    inplace=True
)

In [ ]:
flow_df

In [ ]:
flow_df.rename(
    columns={
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Tagesmittelwert": "Flow River",
    },
    inplace=True,
)

# Preprocess Data

## Ground Water Dataset

In [ ]:
translator = Translator()

In [ ]:
parameters = ts_gw_df["Parameter"].unique()

In [ ]:
parameters

In [ ]:
translator.translate('Aluminium', src="de", dest="en")

In [ ]:
parameters_translated = [translator.translate(item, dest='en').text for item in parameters.tolist()]

In [ ]:
"""Cumulated rainfall
-Environmental temperature
-Water temperature
-Conductivity
-Flow river
Turbidity
-Absorbance 254 nm
-Ammonium
Dissolved oxygen
-Nitrate
-pH
Redox potential"""

In [ ]:
parameters_translated

### Build Dataset per Station

In [ ]:
variables = {
    'Temperatur (Luft)': 'Air Temperature',
    'Temperatur (Wasser)': 'Water Temperature',
    'UV-Adsorption (254)': 'Absorbance 254nm',
    'Leitfähigkeit 25°C vor Ort': 'Conductivity',
    'Ammonium (N)': 'Ammonium',
    'Nitrat (N)': 'Nitrate',
    'pH-Wert (Feld)': 'pH',
}

In [ ]:
ground_df = ts_gw_df[ts_gw_df['Parameter'].isin(variables.keys())]

ground_df['Parameter'] = ground_df['Parameter'].map(variables)

In [ ]:
ground_df['Station ID'].unique()

In [ ]:
ground_df['DateTime'] = pd.to_datetime(ground_df['DateTime'])

In [ ]:
stations_dict = {}
for station in ground_df['Station ID'].unique():
    station_df = ground_df[ground_df['Station ID'] == station]
    station_df = station_df.pivot_table(
        index=pd.Grouper('DateTime'),
        columns='Parameter',
        values='Value'
    )
    
    stations_dict[station] = station_df

### Analyze Stations

In [ ]:
ground_info_df = pd.DataFrame(
    index=pd.Index(
        ['N Samples', '% Missing Values', 'Frequency (days)', 'Start Date', 'End Date'], name='Info'
    ),
    columns=pd.MultiIndex.from_product(
        [ground_df['Station ID'].unique(), variables.values()],
        names=['Station ID', 'Parameter']
    )
)

#### 5130 - Treptow-Köpenick

In [ ]:
station_df = stations_dict[5130]

In [ ]:
station_df.isna().sum() / station_df.shape[0]

In [ ]:
# compute the frequency of the time series
station_df.index.to_series().diff().value_counts()

In [ ]:
# most of the time series have a frequency of 6 months

##### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 5130",
        labels={
            'Date': 'Date',
            column: column
        }
    )
    fig.show()

##### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    
    fig = go.Figure()
    column_df = station_df[column]
    
    for year in column_df.index.year.unique():
        fig.add_trace(
            go.Box(
                y=column_df[column_df.index.year == year],
                name=year
            )
        )
    fig.update_layout(
        title=f"{column} at station 5130",
        xaxis_title="Year",
        yaxis_title=column
    )
    
    fig.show() 

##### Store Info

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    ground_info_df.loc['N Samples', (5130, column)] = station_df[column].dropna().shape[0]
    ground_info_df.loc['% Missing Values', (5130, column)] = station_df[column].isna().sum() / station_df[column].shape[0]
    ground_info_df.loc['Frequency (days)', (5130, column)] = station_df.index.to_series().diff().value_counts().index[0].days
    ground_info_df.loc['Start Date', (5130, column)] = station_df[column].dropna().index.min().strftime("%Y-%m-%d")
    ground_info_df.loc['End Date', (5130, column)] = station_df[column].dropna().index.max().strftime("%Y-%m-%d")

#### 7285 - Steglitz-Zehlendorf

In [ ]:
station_df = stations_dict[7285]

In [ ]:
station_df.isna().sum() / station_df.shape[0]

In [ ]:
# compute the frequency of the time series
station_df.index.to_series().diff().value_counts()

In [ ]:
# most of the time series have a frequency of 6 months

##### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 7285",
        labels={
            'DateTime': 'DateTime',
            column: column
        }
    )
    fig.show()

##### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    
    fig = go.Figure()
    column_df = station_df[column]
    
    for year in column_df.index.year.unique():
        fig.add_trace(
            go.Box(
                y=column_df[column_df.index.year == year],
                name=year
            )
        )
    fig.update_layout(
        title=f"{column} at station 7285",
        xaxis_title="Year",
        yaxis_title=column
    )
    
    fig.show() 

##### Store Info

In [525]:
# store the information in the station_info_df
for column in station_df.columns:
    ground_info_df.loc['N Samples', (7285, column)] = station_df[column].dropna().shape[0]
    ground_info_df.loc['% Missing Values', (7285, column)] = station_df[column].isna().sum() / station_df[column].shape[0]
    ground_info_df.loc['Frequency (days)', (7285, column)] = station_df.index.to_series().diff().value_counts().index[0].days
    ground_info_df.loc['Start Date', (7285, column)] = station_df[column].dropna().index.min().strftime("%Y-%m-%d")
    ground_info_df.loc['End Date', (7285, column)] = station_df[column].dropna().index.max().strftime("%Y-%m-%d")

## Surface Water Dataset

In [ ]:
translator = Translator()

In [ ]:
parameters = ts_sw_df["Parameter"].unique()

In [ ]:
parameters

In [ ]:
parameters_translated = [translator.translate(item, dest='en').text for item in parameters.tolist()]

In [ ]:
"""
Cumulated rainfall
-Environmental temperature
-Water temperature
-Conductivity
-Flow river
Turbidity
-Absorbance 254 nm
-Ammonium
-Dissolved oxygen
-Nitrate
-pH
Redox potential
"""

In [ ]:
parameters_translated

### Build Dataset per Station

NB: there is only one station (305) for which there are measurements of flow close to it (5815911)

In [ ]:
# the parameters that are present for the moment are:
variables = {
    'Lufttemperatur': 'Air Temperature',
    'Wassertemperatur': 'Water Temperature',
    'Spektraler Absorptionskoeffizient (SAK) 254nm': 'Absorbance 254nm',
    'Leitfähigkeit': 'Conductivity',
    'Ammonium-Stickstoff': 'Ammonium',
    'Nitrat-Stickstoff': 'Nitrate',
    'pH-Wert': 'pH',
    'DOC (Gelöster organischer Kohlenstoff)': 'DOC',
    'TOC (Organischer Kohlenstoff)': 'TOC',
    'Sauerstoff-Gehalt': 'Dissolved Oxygen',
}

In [ ]:
surface_df = ts_sw_df[ts_sw_df['Parameter'].isin(variables.keys())]

surface_df['Parameter'] = surface_df['Parameter'].map(variables)

In [ ]:
surface_df['Station ID'].unique()

In [ ]:
surface_df['Station'].unique()

In [ ]:
surface_df['DateTime'] = pd.to_datetime(surface_df['DateTime'])

In [ ]:
stations_dict = {}
for station in surface_df['Station ID'].unique():
    station_df = surface_df[surface_df['Station ID'] == station]
    station_df = station_df.pivot_table(
        index=pd.Grouper('DateTime'),
        columns='Parameter',
        values='Value'
    )
    
    stations_dict[station] = station_df

### Analyze Stations

In [ ]:
surface_info_df = pd.DataFrame(
    index=pd.Index(
        ['N Samples', '% Missing Values', 'Frequency (days)', 'Start Date', 'End Date'], name='Info'
    ),
    columns=pd.MultiIndex.from_product(
        [surface_df['Station ID'].unique(), variables.values()],
        names=['Station ID', 'Parameter']
    )
)

#### 105 - Dämeritzsee-Seemitte

In [ ]:
station_df = stations_dict[105]

In [ ]:
station_df.isna().sum() / station_df.shape[0]

In [ ]:
# compute the frequency of the time series
station_df.index.to_series().diff().value_counts()

In [ ]:
# most of the time series have a frequency of 14 days or 1 month

##### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 105",
        labels={
            'Date': 'Date',
            column: column
        }
    )
    fig.show()

##### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    
    fig = go.Figure()
    column_df = station_df[column]
    
    for year in column_df.index.year.unique():
        fig.add_trace(
            go.Box(
                y=column_df[column_df.index.year == year],
                name=year
            )
        )
    fig.update_layout(
        title=f"{column} at station 105",
        xaxis_title="Year",
        yaxis_title=column
    )
    
    fig.show() 

##### Store Info

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    surface_info_df.loc['N Samples', (105, column)] = station_df[column].dropna().shape[0]
    surface_info_df.loc['% Missing Values', (105, column)] = station_df[column].isna().sum() / station_df[column].shape[0]
    surface_info_df.loc['Frequency (days)', (105, column)] = station_df.index.to_series().diff().value_counts().index[0].days
    surface_info_df.loc['Start Date', (105, column)] = station_df[column].dropna().index.min().strftime("%Y-%m-%d")
    surface_info_df.loc['End Date', (105, column)] = station_df[column].dropna().index.max().strftime("%Y-%m-%d")

#### 305 - Oberhavel-Konradshöhe

In [ ]:
station_df = stations_dict[305]

In [ ]:
flow_df['DateTime'] = pd.to_datetime(flow_df['DateTime'])

station_flow_df = flow_df[flow_df['Station ID'] == 5815911]

station_flow_df = station_flow_df[['DateTime', 'Flow River']].set_index('DateTime')

station_flow_df.index = station_flow_df.index.date
station_df.index = station_df.index.date

# merge the flow data with the surface water data for the same date (just date, not time)
station_df = station_df.merge(
    station_flow_df,
    left_index=True,
    right_index=True,
    how='left'
)

station_df.index = pd.to_datetime(station_df.index)

In [ ]:
station_df.isna().sum() / station_df.shape[0]

In [ ]:
# compute the frequency of the time series
station_df.index.to_series().diff().value_counts()

In [ ]:
# most of the time series have a frequency of 14 days or 1 month

##### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    
    # compute date range for which the data is available
    date_range = station_df[column].dropna().index
    date_range = date_range.min(), date_range.max()
    
    
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 305 - Range: {date_range[0].date()} - {date_range[1].date()}",
        labels={
            'DateTime': 'DateTime',
            column: column
        }
    )
    fig.show()

##### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    
    fig = go.Figure()
    column_df = station_df[column]
    
    for year in column_df.index.year.unique():
        fig.add_trace(
            go.Box(
                y=column_df[column_df.index.year == year],
                name=year
            )
        )
    fig.update_layout(
        title=f"{column} at station 305",
        xaxis_title="Year",
        yaxis_title=column
    )
    
    fig.show() 

##### Store Info

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    surface_info_df.loc['N Samples', (305, column)] = station_df[column].dropna().shape[0]
    surface_info_df.loc['% Missing Values', (305, column)] = station_df[column].isna().sum() / station_df[column].shape[0]
    surface_info_df.loc['Frequency (days)', (305, column)] = station_df.index.to_series().diff().value_counts().index[0].days
    surface_info_df.loc['Start Date', (305, column)] = station_df[column].dropna().index.min().strftime("%Y-%m-%d")
    surface_info_df.loc['End Date', (305, column)] = station_df[column].dropna().index.max().strftime("%Y-%m-%d")

#### 325 - Havel-Pichelsdorfer Gemünd

In [ ]:
station_df = stations_dict[325]

In [ ]:
station_df.isna().sum() / station_df.shape[0]

In [ ]:
# compute the frequency of the time series
station_df.index.to_series().diff().value_counts()

In [ ]:
# most of the time series have a frequency of 14 days or 1 month

##### Time series

In [ ]:
# plot the data
for column in station_df.columns:
    fig = px.line(
        station_df,
        x=station_df.index,
        y=column,
        title=f"{column} at station 325",
        labels={
            'DateTime': 'DateTime',
            column: column
        }
    )
    fig.show()

##### Boxplots

In [ ]:
# boxplot of the data
for column in station_df.columns:
    
    fig = go.Figure()
    column_df = station_df[column]
    
    for year in column_df.index.year.unique():
        fig.add_trace(
            go.Box(
                y=column_df[column_df.index.year == year],
                name=year
            )
        )
    fig.update_layout(
        title=f"{column} at station 325",
        xaxis_title="Year",
        yaxis_title=column
    )
    
    fig.show()

##### Store Info

In [ ]:
# store the information in the station_info_df
for column in station_df.columns:
    surface_info_df.loc['N Samples', (325, column)] = station_df[column].dropna().shape[0]
    surface_info_df.loc['% Missing Values', (325, column)] = station_df[column].isna().sum() / station_df[column].shape[0]
    surface_info_df.loc['Frequency (days)', (325, column)] = station_df.index.to_series().diff().value_counts().index[0].days
    surface_info_df.loc['Start Date', (325, column)] = station_df[column].dropna().index.min().strftime("%Y-%m-%d")
    surface_info_df.loc['End Date', (325, column)] = station_df[column].dropna().index.max().strftime("%Y-%m-%d")

### Store Results

In [ ]:
%%script false --no-raise-error
surface_info_df.to_excel(os.path.join(data_info_folder, "surface_water_info.xlsx"))
ground_info_df.to_excel(os.path.join(data_info_folder, "ground_water_info.xlsx"))

# UVA254 Raw Analysis

In [ ]:
ts_uva254_df = ts_gw_df[
    ts_gw_df["Parameter"] == "UV-Adsorption (254)"
].copy()

In [ ]:
ts_uva254_df

In [ ]:
ts_uva254_df["DateTime"] = pd.to_datetime(
    ts_uva254_df["DateTime"], format="%Y-%m-%d", errors="coerce"
)

In [ ]:
ts_uva254_df['Year'] = ts_uva254_df['DateTime'].dt.year
ts_uva254_df['Month'] = ts_uva254_df['DateTime'].dt.month

In [ ]:
counts = ts_uva254_df['Station ID'].value_counts()

In [ ]:
fig = px.line(
    ts_uva254_df,
    x="Date",
    y="Value",
    color="Station ID",
)

fig.update_layout(
    title={
        "text": "UV-Adsorption (254)",
        "x": 0.5,
        "xanchor": "center",
    },
    xaxis_title="Date",
    yaxis_title="Value",
)

fig.show()

### Station 7285

In [ ]:
station_7285_df = ts_uva254_df[ts_uva254_df['Station ID'] == 7285].copy()

In [ ]:
station_7285_df["Season"] = station_7285_df["Month"].apply(
    lambda x: "Winter"
    if x in [12, 1, 2]
    else "Spring"
    if x in [3, 4, 5]
    else "Summer"
    if x in [6, 7, 8]
    else "Autumn"
)

In [ ]:
# plot station 7285 with seasons as hue
fig = px.line(
    station_7285_df,
    x="DateTime",
    y="Value",
    color="Season",
)

fig.update_layout(
    title={
        "text": "UV-Adsorption (254) at station 7285",
        "x": 0.5,
        "xanchor": "center",
    },
    xaxis_title="DateTime",
    yaxis_title="Value",
)

fig.show()

In [ ]:
mean_station_7285_df = station_7285_df.groupby(["Year"]).agg({"Value": ["mean", "count"]}).reset_index().copy()

In [ ]:
mean_station_7285_df

In [ ]:
mean_station_7285_df = station_7285_df.groupby(["Season"]).agg({"Value": ["mean", "count"]}).reset_index().copy()

In [ ]:
mean_station_7285_df

In [ ]:
# maggiorparte delle misurazioni in autunno e primavera, semestrali circa

In [ ]:
ts = station_7285_df[['DateTime', 'Value']].copy()

result_7285 = smt.seasonal_decompose(
    ts.set_index('Date'), model="additive", period=2
)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=result_7285.trend.index,
        y=result_7285.trend,
        mode="lines+markers",
        name="MA period=2",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=ts['Date'],
        y=ts['Value'],
        mode="lines+markers",
        name="Original",
        line=dict(color="red"),
    )
)

fig.show()

### Station 5130